In [1]:
# data science libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# loading linear algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# tree based algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# machine learning processing and metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

In [2]:
# pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

In [3]:
# Load in my Datasets
train_df = pd.read_csv('Resources/2019loans.csv')
test_df = pd.read_csv('Resources/2020Q1loans.csv')

In [4]:
# Explore train data
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,29.99,0.0,0.0,15.0,0.0,39728.0,38.0,w,12804.66,12804.66,953.33,953.33,570.34,382.99,0.0,0.0,0.0,483.34,0.0,1.0,Individual,0.0,0.0,465790.0,2.0,7.0,3.0,7.0,7.0,82290.0,70.0,1.0,1.0,24683.0,69.0,60200.0,1.0,12.0,2.0,9.0,31053.0,7317.0,77.1,0.0,0.0,160.0,350.0,3.0,3.0,1.0,3.0,8.0,0.0,1.0,4.0,2.0,5.0,18.0,7.0,17.0,4.0,15.0,0.0,0.0,0.0,5.0,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,11.26,2.0,0.0,16.0,0.0,9585.0,20.0,w,15152.97,15152.97,10037.02,10037.02,5847.03,4189.99,0.0,0.0,0.0,478.68,0.0,1.0,Individual,0.0,0.0,60769.0,0.0,2.0,0.0,1.0,17.0,18311.0,52.0,0.0,4.0,4311.0,50.0,20900.0,1.0,0.0,1.0,5.0,3798.0,6990.0,56.0,0.0,0.0,165.0,142.0,13.0,13.0,1.0,17.0,10.0,1.0,5.0,8.0,6.0,7.0,5.0,13.0,14.0,8.0,16.0,0.0,0.0,0.0,0.0,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,11.28,0.0,0.0,12.0,0.0,16708.0,36.0,w,15724.23,15724.23,6971.52,6971.52,4275.77,2695.75,0.0,0.0,0.0,448.95,0.0,1.0,Individual,0.0,0.0,617946.0,1.0,7.0,1.0,2.0,5.0,97335.0,108.0,0.0,0.0,11026.0,98.0,25900.0,2.0,0.0,2.0,3.0,51496.0,5892.0,73.9,0.0,0.0,254.0,239.0,30.0,5.0,3.0,173.0,5.0,1.0,3.0,3.0,3.0,5.0,26.0,4.0,7.0,3.0,12.0,0.0,0.0,0.0,1.0,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,18.08,0.0,0.0,12.0,1.0,8809.0,19.0,w,2930.77,2930.77,81.63,81.63,69.23,12.40,0.0,0.0,0.0,85.76,1.0,1.0,Individual,0.0,568.0,20761.0,2.0,4.0,2.0,2.0,8.0,11952.0,78.0,2.0,4.0,2409.0,49.0,26600.0,4.0,4.0,2.0,6.0,1730.0,13470.0,32.3,0.0,0.0,135.0,146.0,2.0,2.0,0.0,2.0,8.0,0.0,5.0,6.0,6.0,6.0,8.0,8.0,11.0,6.0,12.0,0.0,0.0,0.0,4.0,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,27.77,0.0,2.0,13.0,0.0,65420.0,51.0,w,20778.03,20778.03,14763.99,14763.99,9221.97,5542.02,0.0,0.0,0.0,1056.49,0.0,1.0,Individual,0.0,0.0,237839.0,2.0,3.0,1.0,3.0,5.0,43636.0,74.0,1.0,1.0,19804.0,74.0,88100.0,2.0,11.0,3.0,4.0,18295.0,18517.0,76.7,0.0,0.0,174.0,369.0,1.0,1.0,2.0,34.0,0.0,0.0,6.0,9.0,6.0,19.0,17.0,9.0,32.0,9.0,13.0,0.0,0.0,0.0,2.0,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [5]:
# remove some bad columns from both data sets
train_df.drop(["Unnamed: 0", "index"], axis=1, inplace=True)
test_df.drop(["Unnamed: 0", "index"], axis=1, inplace=True)

In [6]:
#target is loan_status 
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12180 entries, 0 to 12179
Data columns (total 84 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   12180 non-null  float64
 1   int_rate                    12180 non-null  float64
 2   installment                 12180 non-null  float64
 3   home_ownership              12180 non-null  object 
 4   annual_inc                  12180 non-null  float64
 5   verification_status         12180 non-null  object 
 6   loan_status                 12180 non-null  object 
 7   pymnt_plan                  12180 non-null  object 
 8   dti                         12180 non-null  float64
 9   delinq_2yrs                 12180 non-null  float64
 10  inq_last_6mths              12180 non-null  float64
 11  open_acc                    12180 non-null  float64
 12  pub_rec                     12180 non-null  float64
 13  revol_bal                   121

In [7]:
#first indentify the string columns from train_df besides target

str_cols=[]

for col in train_df.columns:
    if (train_df[col].dtype == object) & (col != "loan_status"):
        print(col)
        str_cols.append(col)
        
df_str=train_df.loc[:,str_cols]   
df_str.head()

home_ownership
verification_status
pymnt_plan
initial_list_status
application_type
hardship_flag
debt_settlement_flag


,home_ownership,verification_status,pymnt_plan,initial_list_status,application_type,hardship_flag,debt_settlement_flag
0,MORTGAGE,Not Verified,n,w,Individual,N,N
1,MORTGAGE,Source Verified,n,w,Individual,N,N
2,MORTGAGE,Source Verified,n,w,Individual,N,N
3,RENT,Not Verified,n,w,Individual,N,N
4,MORTGAGE,Source Verified,n,w,Individual,N,N


In [8]:
#decide what type of encoding and if to remove any columns
for col in df_str.columns:
    print(col)
    print(df_str[col].value_counts())
    print()

home_ownership
MORTGAGE    5800
RENT        4944
OWN         1371
ANY           65
Name: home_ownership, dtype: int64

verification_status
Not Verified       5301
Source Verified    4881
Verified           1998
Name: verification_status, dtype: int64

pymnt_plan
n    12180
Name: pymnt_plan, dtype: int64

initial_list_status
w    11158
f     1022
Name: initial_list_status, dtype: int64

application_type
Individual    10400
Joint App      1780
Name: application_type, dtype: int64

hardship_flag
N    11832
Y      348
Name: hardship_flag, dtype: int64

debt_settlement_flag
N    12175
Y        5
Name: debt_settlement_flag, dtype: int64



In [9]:
#dropped columns that are useless due to only one category 
df_str.drop(["pymnt_plan","debt_settlement_flag"] ,axis=1, inplace=True)

In [10]:
#label encode other columns
df_le = df_str.loc[: ,["initial_list_status","application_type","hardship_flag"]]
df_le["initial_list_status"] = [1 if x == "f" else 0 for x in df_le.initial_list_status]
df_le["application_type"] = [1 if x == "Joint App" else 0 for x in df_le.application_type]
df_le["hardship_flag"] = [1 if x == "Y" else 0 for x in df_le.hardship_flag]

df_le.head()

,initial_list_status,application_type,hardship_flag
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [11]:
#onehot encode the rest 
df_oh= pd.get_dummies(df_str.loc[:, ["home_ownership", "verification_status"]])
df_oh.head()

,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,0,1,0,0,1,0,0
1,0,1,0,0,0,1,0
2,0,1,0,0,0,1,0
3,0,0,0,1,1,0,0
4,0,1,0,0,0,1,0


In [12]:
#Second indentify the numeric columns from train_df
num_cols=[]

for col in train_df.columns:
    if (train_df[col].dtype != object) & (col != "loan_status"):
        print(col)
        num_cols.append(col)
        
df_num=train_df.loc[:,num_cols]   
df_num.head()

loan_amnt
int_rate
installment
annual_inc
dti
delinq_2yrs
inq_last_6mths
open_acc
pub_rec
revol_bal
total_acc
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_amnt
collections_12_mths_ex_med
policy_code
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
bc_util
chargeoff_within_12_mths
delinq_amnt
mo_sin_old_il_acct
mo_sin_old_rev_tl_op
mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_tl
mort_acc
mths_since_recent_bc
mths_since_recent_inq
num_accts_ever_120_pd
num_actv_bc_tl
num_actv_rev_tl
num_bc_sats
num_bc_tl
num_il_tl
num_op_rev_tl
num_rev_accts
num_rev_tl_bal_gt_0
num_sats
num_tl_120dpd_2m
num_tl_30dpd
num_tl_90g_dpd_24m
num_tl_op_past_12m
pct_tl_nvr_dlq
percent_bc_gt_75
pub_rec_bankru

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,13375.0,0.1797,483.34,223000.0,29.99,0.0,0.0,15.0,0.0,39728.0,38.0,12804.66,12804.66,953.33,953.33,570.34,382.99,0.0,0.0,0.0,483.34,0.0,1.0,0.0,0.0,465790.0,2.0,7.0,3.0,7.0,7.0,82290.0,70.0,1.0,1.0,24683.0,69.0,60200.0,1.0,12.0,2.0,9.0,31053.0,7317.0,77.1,0.0,0.0,160.0,350.0,3.0,3.0,1.0,3.0,8.0,0.0,1.0,4.0,2.0,5.0,18.0,7.0,17.0,4.0,15.0,0.0,0.0,0.0,5.0,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0
1,21000.0,0.1308,478.68,123000.0,11.26,2.0,0.0,16.0,0.0,9585.0,20.0,15152.97,15152.97,10037.02,10037.02,5847.03,4189.99,0.0,0.0,0.0,478.68,0.0,1.0,0.0,0.0,60769.0,0.0,2.0,0.0,1.0,17.0,18311.0,52.0,0.0,4.0,4311.0,50.0,20900.0,1.0,0.0,1.0,5.0,3798.0,6990.0,56.0,0.0,0.0,165.0,142.0,13.0,13.0,1.0,17.0,10.0,1.0,5.0,8.0,6.0,7.0,5.0,13.0,14.0,8.0,16.0,0.0,0.0,0.0,0.0,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0
2,20000.0,0.1240,448.95,197000.0,11.28,0.0,0.0,12.0,0.0,16708.0,36.0,15724.23,15724.23,6971.52,6971.52,4275.77,2695.75,0.0,0.0,0.0,448.95,0.0,1.0,0.0,0.0,617946.0,1.0,7.0,1.0,2.0,5.0,97335.0,108.0,0.0,0.0,11026.0,98.0,25900.0,2.0,0.0,2.0,3.0,51496.0,5892.0,73.9,0.0,0.0,254.0,239.0,30.0,5.0,3.0,173.0,5.0,1.0,3.0,3.0,3.0,5.0,26.0,4.0,7.0,3.0,12.0,0.0,0.0,0.0,1.0,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0
3,3000.0,0.1240,100.22,45000.0,18.08,0.0,0.0,12.0,1.0,8809.0,19.0,2930.77,2930.77,81.63,81.63,69.23,12.40,0.0,0.0,0.0,85.76,1.0,1.0,0.0,568.0,20761.0,2.0,4.0,2.0,2.0,8.0,11952.0,78.0,2.0,4.0,2409.0,49.0,26600.0,4.0,4.0,2.0,6.0,1730.0,13470.0,32.3,0.0,0.0,135.0,146.0,2.0,2.0,0.0,2.0,8.0,0.0,5.0,6.0,6.0,6.0,8.0,8.0,11.0,6.0,12.0,0.0,0.0,0.0,4.0,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0
4,30000.0,0.1612,1056.49,133000.0,27.77,0.0,2.0,13.0,0.0,65420.0,51.0,20778.03,20778.03,14763.99,14763.99,9221.97,5542.02,0.0,0.0,0.0,1056.49,0.0,1.0,0.0,0.0,237839.0,2.0,3.0,1.0,3.0,5.0,43636.0,74.0,1.0,1.0,19804.0,74.0,88100.0,2.0,11.0,3.0,4.0,18295.0,18517.0,76.7,0.0,0.0,174.0,369.0,1.0,1.0,2.0,34.0,0.0,0.0,6.0,9.0,6.0,19.0,17.0,9.0,32.0,9.0,13.0,0.0,0.0,0.0,2.0,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0


In [13]:
#scaler
scaler = StandardScaler().fit(df_num)
df_scale = pd.DataFrame(scaler.transform(df_num), columns=df_num.columns)
df_scale.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,-0.393112,0.736585,-0.087609,0.790279,0.309615,-0.320026,-0.695401,0.376917,-0.351986,0.979339,1.083336,-0.071368,-0.071217,-1.093789,-1.093659,-0.924224,-0.968314,-0.150981,0.0,0.0,0.053954,-0.133126,0.0,-0.009061,-0.139714,1.806319,0.785406,1.179572,2.140917,3.136183,-0.516428,0.741144,-0.020749,-0.225280,-0.669299,3.455342,0.616623,0.503602,-0.282801,3.506591,-0.132295,1.186045,1.177360,-0.430820,0.951911,-0.071082,-0.009061,0.601757,1.907460,-0.648755,-0.648350,-0.175635,-0.661807,0.131913,-0.359420,-1.167625,-0.523893,-0.989682,-0.509289,1.071847,-0.334151,0.465341,-0.525210,0.379269,0.0,-0.009061,-0.148869,1.326667,0.617581,0.500053,-0.351986,0.0,1.965731,1.010651,0.156409,2.188821
1,0.351681,-0.191716,-0.103427,0.196561,-0.473994,2.170653,-0.695401,0.539402,-0.351986,-0.354346,-0.350335,0.195949,0.196078,0.736269,0.736426,0.511144,0.847978,-0.150981,0.0,0.0,0.048334,-0.133126,0.0,-0.009061,-0.139714,-0.551749,-0.871517,-0.435953,-0.881145,-0.610564,0.220799,-0.507964,-0.832495,-0.862659,0.453420,-0.332941,-0.422608,-0.500258,-0.282801,-0.578497,-0.534629,-0.015813,-0.624212,-0.448260,0.199035,-0.071082,-0.009061,0.685436,-0.228403,-0.042949,1.009132,-0.175635,-0.194150,0.467448,0.323961,0.397217,0.610258,0.222235,-0.072617,-0.589139,0.832750,0.090251,0.608016,0.541807,0.0,-0.009061,-0.148869,-1.188713,-1.069844,0.007255,-0.351986,0.0,-0.344265,-0.585044,-0.479179,-0.399905
2,0.254003,-0.320805,-0.204342,0.635913,-0.473157,-0.320026,-0.695401,-0.110538,-0.351986,-0.039187,0.924039,0.260977,0.261102,0.118673,0.118822,0.083729,0.135087,-0.150981,0.0,0.0,0.012484,-0.133126,0.0,-0.009061,-0.139714,2.692184,-0.043055,1.179572,0.126209,0.013894,-0.663873,1.034879,1.692938,-0.862659,-1.043538,0.915749,2.202816,-0.372541,0.342678,-0.578497,-0.132295,-0.616742,2.528656,-0.506821,0.837731,-0.071082,-0.009061,2.174926,0.767648,0.986922,-0.316854,1.018135,5.016891,-0.371388,0.323961,-0.385204,-0.807431,-0.686703,-0.509289,2.093992,-0.917601,-0.784958,-0.808517,-0.108345,0.0,-0.009061,-0.148869,-0.685637,-0.991097,0.007255,-0.351986,0.0,2.230882,0.875447,-0.214679,0.655196
3,-1.406519,-0.320805,-1.388060,-0.266539,-0.188665,-0.320026,-0.695401,-0.110538,2.682121,-0.388681,-0.429984,-1.195351,-1.195107,-1.269408,-1.269280,-1.060536,-1.145120,-0.150981,0.0,0.0,-0.425476,6.485154,0.0,-0.009061,0.289225,-0.784679,0.785406,0.210257,1.133563,0.013894,-0.442705,-0.632115,0.340027,0.412098,0.453420,-0.686628,-0.477304,-0.354660,1.593635,0.783199,-0.132295,0.284652,-0.760909,-0.102659,-0.646613,-0.071082,-0.009061,0.183361,-0.187329,-0.709335,-0.814099,-0.772520,-0.695211,0.131913,-0.359420,0.397217,0.043182,0.222235,-0.290953,-0.205835,-0.139667,-0.284838,0.041403,-0.108345,0.0,-0.009061,-0.148869,0.823591,0.617581,-0.482592,2.682121,0.0,-0.815953,-0.706007,-0.321269,-0.783830
4,1.230781,0.385387,1.857873,0.255933,0.216736,-0.320026,1.759668,0.051947,-0.351986,2.1160

In [14]:
#concatenate all out columns 
df_train = pd.concat([df_scale, df_le, df_oh,], axis=1)

#addback in the target
df_train["loan_status"] = train_df.loan_status

#make the target numeric aswell
df_train["loan_status"] = [1 if x== "high_risk" else 0 for x in df_train.loan_status]
df_train.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,initial_list_status,application_type,hardship_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,loan_status
0,-0.393112,0.736585,-0.087609,0.790279,0.309615,-0.320026,-0.695401,0.376917,-0.351986,0.979339,1.083336,-0.071368,-0.071217,-1.093789,-1.093659,-0.924224,-0.968314,-0.150981,0.0,0.0,0.053954,-0.133126,0.0,-0.009061,-0.139714,1.806319,0.785406,1.179572,2.140917,3.136183,-0.516428,0.741144,-0.020749,-0.225280,-0.669299,3.455342,0.616623,0.503602,-0.282801,3.506591,-0.132295,1.186045,1.177360,-0.430820,0.951911,-0.071082,-0.009061,0.601757,1.907460,-0.648755,-0.648350,-0.175635,-0.661807,0.131913,-0.359420,-1.167625,-0.523893,-0.989682,-0.509289,1.071847,-0.334151,0.465341,-0.525210,0.379269,0.0,-0.009061,-0.148869,1.326667,0.617581,0.500053,-0.351986,0.0,1.965731,1.010651,0.156409,2.188821,0,0,0,0,1,0,0,1,0,0,0
1,0.351681,-0.191716,-0.103427,0.196561,-0.473994,2.170653,-0.695401,0.539402,-0.351986,-0.354346,-0.350335,0.195949,0.196078,0.736269,0.736426,0.511144,0.847978,-0.150981,0.0,0.0,0.048334,-0.133126,0.0,-0.009061,-0.139714,-0.551749,-0.871517,-0.435953,-0.881145,-0.610564,0.220799,-0.507964,-0.832495,-0.862659,0.453420,-0.332941,-0.422608,-0.500258,-0.282801,-0.578497,-0.534629,-0.015813,-0.624212,-0.448260,0.199035,-0.071082,-0.009061,0.685436,-0.228403,-0.042949,1.009132,-0.175635,-0.194150,0.467448,0.323961,0.397217,0.610258,0.222235,-0.072617,-0.589139,0.832750,0.090251,0.608016,0.541807,0.0,-0.009061,-0.148869,-1.188713,-1.069844,0.007255,-0.351986,0.0,-0.344265,-0.585044,-0.479179,-0.399905,0,0,0,0,1,0,0,0,1,0,0
2,0.254003,-0.320805,-0.204342,0.635913,-0.473157,-0.320026,-0.695401,-0.110538,-0.351986,-0.039187,0.924039,0.260977,0.261102,0.118673,0.118822,0.083729,0.135087,-0.150981,0.0,0.0,0.012484,-0.133126,0.0,-0.009061,-0.139714,2.692184,-0.043055,1.179572,0.126209,0.013894,-0.663873,1.034879,1.692938,-0.862659,-1.043538,0.915749,2.202816,-0.372541,0.342678,-0.578497,-0.132295,-0.616742,2.528656,-0.506821,0.837731,-0.071082,-0.009061,2.174926,0.767648,0.986922,-0.316854,1.018135,5.016891,-0.371388,0.323961,-0.385204,-0.807431,-0.686703,-0.509289,2.093992,-0.917601,-0.784958,-0.808517,-0.108345,0.0,-0.009061,-0.148869,-0.685637,-0.991097,0.007255,-0.351986,0.0,2.230882,0.875447,-0.214679,0.655196,0,0,0,0,1,0,0,0,1,0,0
3,-1.406519,-0.320805,-1.388060,-0.266539,-0.188665,-0.320026,-0.695401,-0.110538,2.682121,-0.388681,-0.429984,-1.195351,-1.195107,-1.269408,-1.269280,-1.060536,-1.145120,-0.150981,0.0,0.0,-0.425476,6.485154,0.0,-0.009061,0.289225,-0.784679,0.785406,0.210257,1.133563,0.013894,-0.442705,-0.632115,0.340027,0.412098,0.453420,-0.686628,-0.477304,-0.354660,1.593635,0.783199,-0.132295,0.284652,-0.760909,-0.102659,-0.646613,-0.071082,-0.009061,0.183361,-0.187329,-0.709335,-0.814099,-0.772520,-0

In [15]:
corrs = abs(df_train.corr())["loan_status"].sort_values(ascending=False)
corrs
#NAN means the number is the same all throughout

loan_status                            1.000000
int_rate                               0.241308
total_rec_int                          0.208996
total_rec_late_fee                     0.124084
out_prncp_inv                          0.101959
out_prncp                              0.101923
installment                            0.095323
acc_open_past_24mths                   0.088339
inq_last_6mths                         0.084456
hardship_flag                          0.081807
num_tl_op_past_12m                     0.080955
mths_since_recent_inq                  0.078746
inq_last_12m                           0.078254
total_bc_limit                         0.077632
open_rv_24m                            0.074387
bc_open_to_buy                         0.072173
mo_sin_old_rev_tl_op                   0.071942
total_rec_prncp                        0.071784
open_acc_6m                            0.071759
open_rv_12m                            0.064837
mo_sin_rcnt_rev_tl_op                  0

In [16]:
# Drop NANs and columns less than 0.01

In [17]:
corrs.index

Index(['loan_status', 'int_rate', 'total_rec_int', 'total_rec_late_fee',
       'out_prncp_inv', 'out_prncp', 'installment', 'acc_open_past_24mths',
       'inq_last_6mths', 'hardship_flag', 'num_tl_op_past_12m',
       'mths_since_recent_inq', 'inq_last_12m', 'total_bc_limit',
       'open_rv_24m', 'bc_open_to_buy', 'mo_sin_old_rev_tl_op',
       'total_rec_prncp', 'open_acc_6m', 'open_rv_12m',
       'mo_sin_rcnt_rev_tl_op', 'max_bal_bc', 'inq_fi', 'last_pymnt_amnt',
       'percent_bc_gt_75', 'home_ownership_MORTGAGE', 'loan_amnt', 'all_util',
       'mths_since_recent_bc', 'total_rev_hi_lim', 'mort_acc',
       'mo_sin_rcnt_tl', 'il_util', 'open_il_24m', 'open_il_12m', 'bc_util',
       'mo_sin_old_il_acct', 'delinq_2yrs', 'verification_status_Verified',
       'home_ownership_RENT', 'pct_tl_nvr_dlq', 'total_pymnt_inv',
       'total_pymnt', 'tot_hi_cred_lim', 'home_ownership_OWN',
       'num_actv_rev_tl', 'num_rev_tl_bal_gt_0',
       'verification_status_Not Verified', 'applicat

In [18]:
cols_drop = ['num_sats', 'open_acc',
       'num_tl_90g_dpd_24m', 'delinq_amnt', 'acc_now_delinq', 'num_tl_30dpd',
       'annual_inc', 'chargeoff_within_12_mths', 'total_il_high_credit_limit',
       'total_bal_ex_mort', 'tot_coll_amt', 'pub_rec',
       'pub_rec_bankruptcies', 'total_acc', 'num_il_tl', 'recoveries',
       'collection_recovery_fee', 'policy_code', 'num_tl_120dpd_2m',
       'tax_liens']

cols_keep = [x for x in df_train if x not in cols_drop ]
df_train = df_train.loc[:, cols_keep]
df_train.head()

,loan_amnt,int_rate,installment,dti,delinq_2yrs,inq_last_6mths,revol_bal,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,tot_hi_cred_lim,total_bc_limit,initial_list_status,application_type,hardship_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,loan_status
0,-0.393112,0.736585,-0.087609,0.309615,-0.320026,-0.695401,0.979339,-0.071368,-0.071217,-1.093789,-1.093659,-0.924224,-0.968314,-0.150981,0.053954,-0.133126,1.806319,0.785406,1.179572,2.140917,3.136183,-0.516428,0.741144,-0.020749,-0.225280,-0.669299,3.455342,0.616623,0.503602,-0.282801,3.506591,-0.132295,1.186045,1.177360,-0.430820,0.951911,0.601757,1.907460,-0.648755,-0.648350,-0.175635,-0.661807,0.131913,-0.359420,-1.167625,-0.523893,-0.989682,-0.509289,-0.334151,0.465341,-0.525210,1.326667,0.617581,0.500053,1.965731,0.156409,0,0,0,0,1,0,0,1,0,0,0
1,0.351681,-0.191716,-0.103427,-0.473994,2.170653,-0.695401,-0.354346,0.195949,0.196078,0.736269,0.736426,0.511144,0.847978,-0.150981,0.048334,-0.133126,-0.551749,-0.871517,-0.435953,-0.881145,-0.610564,0.220799,-0.507964,-0.832495,-0.862659,0.453420,-0.332941,-0.422608,-0.500258,-0.282801,-0.578497,-0.534629,-0.015813,-0.624212,-0.448260,0.199035,0.685436,-0.228403,-0.042949,1.009132,-0.175635,-0.194150,0.467448,0.323961,0.397217,0.610258,0.222235,-0.072617,0.832750,0.090251,0.608016,-1.188713,-1.069844,0.007255,-0.344265,-0.479179,0,0,0,0,1,0,0,0,1,0,0
2,0.254003,-0.320805,-0.204342,-0.473157,-0.320026,-0.695401,-0.039187,0.260977,0.261102,0.118673,0.118822,0.083729,0.135087,-0.150981,0.012484,-0.133126,2.692184,-0.043055,1.179572,0.126209,0.013894,-0.663873,1.034879,1.692938,-0.862659,-1.043538,0.915749,2.202816,-0.372541,0.342678,-0.578497,-0.132295,-0.616742,2.528656,-0.506821,0.837731,2.174926,0.767648,0.986922,-0.316854,1.018135,5.016891,-0.371388,0.323961,-0.385204,-0.807431,-0.686703,-0.509289,-0.917601,-0.784958,-0.808517,-0.685637,-0.991097,0.007255,2.230882,-0.214679,0,0,0,0,1,0,0,0,1,0,0
3,-1.406519,-0.320805,-1.388060,-0.188665,-0.320026,-0.695401,-0.388681,-1.195351,-1.195107,-1.269408,-1.269280,-1.060536,-1.145120,-0.150981,-0.425476,6.485154,-0.784679,0.785406,0.210257,1.133563,0.013894,-0.442705,-0.632115,0.340027,0.412098,0.453420,-0.686628,-0.477304,-0.354660,1.593635,0.783199,-0.132295,0.284652,-0.760909,-0.102659,-0.646613,0.183361,-0.187329,-0.709335,-0.814099,-0.772520,-0.695211,0.131913,-0.359420,0.397217,0.043182,0.222235,-0.290953,-0.139667,-0.284838,0.041403,0.823591,0.617581,-0.482592,-0.815953,-0.321269,0,0,0,0,0,0,1,1,0,0,0
4,1.230781,0.385387,1.857873,0.216736,-0.320026,1.759668,2.116088,0.836271,0.836348,1.688594,1.688765,1.429196,1.493022,-0.150981,0.745097,-0.133126,0.479168,0.785406,-0.112848,0.126209,0.638352,-0.663873,-0.013526,0.159639,-0.225280,-0.669299,2.548066,0.890104,1.216265,0.342678,3.166167,0.270040,-0.316277,0.334049,0.166516,0.937639,0.836058,2.102563,-0.769916,-0.979847,0.421250,0.373720,-1.210223,-0.359420,0.788427,0.893796,0.222235,2.547413,0.054816,2.340789,0.891322,-0.182561,0.617581,0.992851,0.438026,2.031592,0,0,0,0,1,0,0,0,1,0,0


In [19]:
#repeat everything we did for train_df to test_df
df_test = test_df.copy()

#first indentify the string columns 
df_str_test = test_df.loc[:,str_cols] 

#dropped columns that are useless due to only one category 
df_str_test.drop(["pymnt_plan","debt_settlement_flag"] ,axis=1, inplace=True)

#label encode other columns
df_le_test = df_str_test.loc[: ,["initial_list_status","application_type","hardship_flag"]]
df_le_test["initial_list_status"] = [1 if x == "f" else 0 for x in df_le_test.initial_list_status]
df_le_test["application_type"] = [1 if x == "Joint App" else 0 for x in df_le_test.application_type]
df_le_test["hardship_flag"] = [1 if x == "Y" else 0 for x in df_le_test.hardship_flag]
# df_le_test.head()

#onehot encode the rest 
df_oh_test= pd.get_dummies(df_str_test.loc[:, ["home_ownership", "verification_status"]])
# df_oh_test.head()

#first indentify the number columns 
df_num_test= df_test.loc[:,num_cols]

#scaler
df_scale_test = pd.DataFrame(scaler.transform(df_num_test), columns=df_num_test.columns)
df_scale_test.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,2.207559,-1.120016,1.037148,0.297493,-0.118797,-0.320026,0.532133,0.864373,-0.351986,-0.359390,1.640875,2.255996,2.255958,0.676429,0.676586,0.756783,0.275448,-0.150981,0.0,0.0,0.453530,-0.133126,0.0,-0.009061,-0.139714,1.293876,-0.043055,0.856467,0.126209,0.013894,-0.368982,0.334133,-0.877593,-0.225280,-0.669299,-0.536190,-1.735320,1.175396,-0.908279,0.102351,-0.534629,-0.616742,0.511991,2.652502,-1.345967,-0.071082,-0.009061,1.137304,0.161803,-0.709335,-0.814099,1.018135,0.707761,-0.874689,-0.35942,0.788427,0.043182,0.222235,0.582390,1.327383,0.443783,1.090490,0.041403,0.866882,0.0,-0.009061,-0.148869,-0.182561,0.358842,-0.975390,-0.351986,0.0,1.710119,0.144260,1.838152,0.830624
1,-1.113486,0.218331,-1.019839,-0.207167,-0.463116,2.170653,-0.695401,-0.760478,-0.351986,-0.721803,-0.987523,-1.528971,-1.528701,-0.039665,-0.039519,0.552755,-1.062493,-0.150981,0.0,0.0,6.432914,-0.133126,0.0,-0.009061,-0.139714,-0.768965,-0.043055,-0.759058,0.126209,-0.610564,-0.590150,-0.432427,1.061579,-0.862659,-0.669299,-1.002009,0.561926,-0.753139,-0.282801,-0.238073,0.270040,-0.917206,-0.681390,-0.571461,-1.060516,-0.071082,-0.009061,0.317247,-0.207866,0.502277,-0.151106,-0.772520,1.208823,-0.203621,-0.35942,-0.776415,-0.807431,-0.989682,-1.164297,-0.844675,-0.334151,-0.534898,-0.808517,-0.758497,0.0,-0.009061,-0.148869,-0.685637,-3.128502,-0.975390,-0.351986,0.0,-0.854304,-0.660250,-0.873954,-0.625935
2,-1.347913,0.542951,-1.292848,-0.284351,-0.663097,-0.320026,-0.695401,-1.085448,-0.351986,-0.567962,-1.465413,-1.528971,-1.528701,-0.559823,-0.559685,-0.100094,-1.149257,-0.150981,0.0,0.0,3.820822,-0.133126,0.0,-0.009061,-0.139714,-0.793866,-0.871517,-0.112848,-0.881145,-1.235022,1.916420,-0.583814,1.151774,-0.225280,-0.295059,-0.250005,1.382371,-0.827215,-0.908279,-0.578497,-0.132295,-0.917206,-0.621635,-0.685435,0.527303,-0.071082,-0.009061,-1.172243,-0.587803,-0.285271,0.346139,-0.772520,-0.461383,-0.035854,-0.35942,-1.167625,-1.374506,-0.989682,-1.164297,-0.844675,-1.112084,-1.285077,-1.375130,-1.083573,0.0,-0.009061,-0.148869,-0.685637,0.617581,-0.975390,-0.351986,0.0,-0.914226,-0.732760,-0.818685,-0.791627
3,0.254003,0.218331,-0.104615,0.060006,-0.437596,-0.320026,1.759668,-0.922963,-0.351986,-0.215151,-0.270687,0.477242,0.477349,-0.325590,-0.325448,-0.433064,-0.010579,-0.150981,0.0,0.0,0.047912,-0.133126,0.0,-0.009061,-0.139714,-0.650445,-0.043055,-0.435953,0.126209,0.638352,-0.221537,-0.258548,0.700803,1.049477,0.079180,-0.034297,1.108890,-0.510476,-0.908279,0.442775,0.270040,0.284652,-0.392530,-0.474607,0.088423,-0.071082,-0.009061,0.133153,1.291345,-0.588174,-0.482602,-0.772520,-0.528191,-0.371388,-0.35942,-0.776415,-0.523893,-0.686703,0.145719,-0.205835,-0.723118,-0.159808,-0.525210,-0.921035,0.0,-0.009061,-0.148869,0.823591,0.617581,0.500053,-0.351986,0.0,-0.740712,-0.315128,-0.562082,-0.388709
4,-1.347913,-0.320805,-1.320003,-0.236853,-0.272339,-0.320026,2.

In [20]:
#concatenate all out columns 
df_test = pd.concat([df_scale_test, df_le_test, df_oh_test,], axis=1)

#addback in the target
df_test["loan_status"] = test_df.loan_status

#make the target numeric aswell
df_test["loan_status"] = [1 if x== "high_risk" else 0 for x in df_test.loan_status]
df_test.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,initial_list_status,application_type,hardship_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,loan_status
0,2.207559,-1.120016,1.037148,0.297493,-0.118797,-0.320026,0.532133,0.864373,-0.351986,-0.359390,1.640875,2.255996,2.255958,0.676429,0.676586,0.756783,0.275448,-0.150981,0.0,0.0,0.453530,-0.133126,0.0,-0.009061,-0.139714,1.293876,-0.043055,0.856467,0.126209,0.013894,-0.368982,0.334133,-0.877593,-0.225280,-0.669299,-0.536190,-1.735320,1.175396,-0.908279,0.102351,-0.534629,-0.616742,0.511991,2.652502,-1.345967,-0.071082,-0.009061,1.137304,0.161803,-0.709335,-0.814099,1.018135,0.707761,-0.874689,-0.35942,0.788427,0.043182,0.222235,0.582390,1.327383,0.443783,1.090490,0.041403,0.866882,0.0,-0.009061,-0.148869,-0.182561,0.358842,-0.975390,-0.351986,0.0,1.710119,0.144260,1.838152,0.830624,0,0,0,0,1,0,0,1,0,0,0
1,-1.113486,0.218331,-1.019839,-0.207167,-0.463116,2.170653,-0.695401,-0.760478,-0.351986,-0.721803,-0.987523,-1.528971,-1.528701,-0.039665,-0.039519,0.552755,-1.062493,-0.150981,0.0,0.0,6.432914,-0.133126,0.0,-0.009061,-0.139714,-0.768965,-0.043055,-0.759058,0.126209,-0.610564,-0.590150,-0.432427,1.061579,-0.862659,-0.669299,-1.002009,0.561926,-0.753139,-0.282801,-0.238073,0.270040,-0.917206,-0.681390,-0.571461,-1.060516,-0.071082,-0.009061,0.317247,-0.207866,0.502277,-0.151106,-0.772520,1.208823,-0.203621,-0.35942,-0.776415,-0.807431,-0.989682,-1.164297,-0.844675,-0.334151,-0.534898,-0.808517,-0.758497,0.0,-0.009061,-0.148869,-0.685637,-3.128502,-0.975390,-0.351986,0.0,-0.854304,-0.660250,-0.873954,-0.625935,0,0,0,0,0,0,1,1,0,0,0
2,-1.347913,0.542951,-1.292848,-0.284351,-0.663097,-0.320026,-0.695401,-1.085448,-0.351986,-0.567962,-1.465413,-1.528971,-1.528701,-0.559823,-0.559685,-0.100094,-1.149257,-0.150981,0.0,0.0,3.820822,-0.133126,0.0,-0.009061,-0.139714,-0.793866,-0.871517,-0.112848,-0.881145,-1.235022,1.916420,-0.583814,1.151774,-0.225280,-0.295059,-0.250005,1.382371,-0.827215,-0.908279,-0.578497,-0.132295,-0.917206,-0.621635,-0.685435,0.527303,-0.071082,-0.009061,-1.172243,-0.587803,-0.285271,0.346139,-0.772520,-0.461383,-0.035854,-0.35942,-1.167625,-1.374506,-0.989682,-1.164297,-0.844675,-1.112084,-1.285077,-1.375130,-1.083573,0.0,-0.009061,-0.148869,-0.685637,0.617581,-0.975390,-0.351986,0.0,-0.914226,-0.732760,-0.818685,-0.791627,0,0,0,0,0,0,1,1,0,0,0
3,0.254003,0.218331,-0.104615,0.060006,-0.437596,-0.320026,1.759668,-0.922963,-0.351986,-0.215151,-0.270687,0.477242,0.477349,-0.325590,-0.325448,-0.433064,-0.010579,-0.150981,0.0,0.0,0.047912,-0.133126,0.0,-0.009061,-0.139714,-0.650445,-0.043055,-0.435953,0.126209,0.638352,-0.221537,-0.258548,0.700803,1.049477,0.079180,-0.034297,1.108890,-0.510476,-0.908279,0.442775,0.270040,0.284652,-0.392530,-0.474607,0.088423,-0.071082,-0.009061,0.133153,1.291345,-0.58817

In [21]:
df_test = df_test.loc[:, cols_keep]
df_test.head()

,loan_amnt,int_rate,installment,dti,delinq_2yrs,inq_last_6mths,revol_bal,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,tot_hi_cred_lim,total_bc_limit,initial_list_status,application_type,hardship_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,loan_status
0,2.207559,-1.120016,1.037148,-0.118797,-0.320026,0.532133,-0.359390,2.255996,2.255958,0.676429,0.676586,0.756783,0.275448,-0.150981,0.453530,-0.133126,1.293876,-0.043055,0.856467,0.126209,0.013894,-0.368982,0.334133,-0.877593,-0.225280,-0.669299,-0.536190,-1.735320,1.175396,-0.908279,0.102351,-0.534629,-0.616742,0.511991,2.652502,-1.345967,1.137304,0.161803,-0.709335,-0.814099,1.018135,0.707761,-0.874689,-0.35942,0.788427,0.043182,0.222235,0.582390,0.443783,1.090490,0.041403,-0.182561,0.358842,-0.975390,1.710119,1.838152,0,0,0,0,1,0,0,1,0,0,0
1,-1.113486,0.218331,-1.019839,-0.463116,2.170653,-0.695401,-0.721803,-1.528971,-1.528701,-0.039665,-0.039519,0.552755,-1.062493,-0.150981,6.432914,-0.133126,-0.768965,-0.043055,-0.759058,0.126209,-0.610564,-0.590150,-0.432427,1.061579,-0.862659,-0.669299,-1.002009,0.561926,-0.753139,-0.282801,-0.238073,0.270040,-0.917206,-0.681390,-0.571461,-1.060516,0.317247,-0.207866,0.502277,-0.151106,-0.772520,1.208823,-0.203621,-0.35942,-0.776415,-0.807431,-0.989682,-1.164297,-0.334151,-0.534898,-0.808517,-0.685637,-3.128502,-0.975390,-0.854304,-0.873954,0,0,0,0,0,0,1,1,0,0,0
2,-1.347913,0.542951,-1.292848,-0.663097,-0.320026,-0.695401,-0.567962,-1.528971,-1.528701,-0.559823,-0.559685,-0.100094,-1.149257,-0.150981,3.820822,-0.133126,-0.793866,-0.871517,-0.112848,-0.881145,-1.235022,1.916420,-0.583814,1.151774,-0.225280,-0.295059,-0.250005,1.382371,-0.827215,-0.908279,-0.578497,-0.132295,-0.917206,-0.621635,-0.685435,0.527303,-1.172243,-0.587803,-0.285271,0.346139,-0.772520,-0.461383,-0.035854,-0.35942,-1.167625,-1.374506,-0.989682,-1.164297,-1.112084,-1.285077,-1.375130,-0.685637,0.617581,-0.975390,-0.914226,-0.818685,0,0,0,0,0,0,1,1,0,0,0
3,0.254003,0.218331,-0.104615,-0.437596,-0.320026,1.759668,-0.215151,0.477242,0.477349,-0.325590,-0.325448,-0.433064,-0.010579,-0.150981,0.047912,-0.133126,-0.650445,-0.043055,-0.435953,0.126209,0.638352,-0.221537,-0.258548,0.700803,1.049477,0.079180,-0.034297,1.108890,-0.510476,-0.908279,0.442775,0.270040,0.284652,-0.392530,-0.474607,0.088423,0.133153,1.291345,-0.588174,-0.482602,-0.772520,-0.528191,-0.371388,-0.35942,-0.776415,-0.523893,-0.686703,0.145719,-0.723118,-0.159808,-0.525210,0.823591,0.617581,0.500053,-0.740712,-0.562082,0,0,0,0,0,0,1,1,0,0,0
4,-1.347913,-0.320805,-1.320003,-0.272339,-0.320026,2.987203,-0.317711,-1.528971,-1.528701,-0.560075,-0.559937,-0.100094,-1.149853,-0.150981,3.818217,-0.133126,-0.716636,0.785406,-0.435953,0.126209,-0.610564,-0.368982,-0.435258,0.790997,0.412098,-0.295059,-0.021652,0.726015,-0.497704,0.968157,-0.238073,4.695721,-0.616742,-0.517790,-0.256419,-0.029325,-1.406545,-0.053837,-0.648755,-0.648350,-0.772520,-0.661807,-0.706922,-0.35942,0.006006,-0.523893,-0.383724,-0.727625,-0.917601,-1.160047,-0.525210,0.320515,0.617581,-0.237668,-0.795312,-0.277843,0,0,0,0,0,0,1,1,0,0,0


In [22]:
# Specify my X and Y
#drop the first two one hot encoded variables and target
X_train = df_train.drop(["loan_status", "home_ownership_ANY", "verification_status_Not Verified"], axis=1).values
y_train = df_train["loan_status"].values

X_test = df_test.drop(["loan_status", "home_ownership_ANY", "verification_status_Not Verified"], axis=1).values
y_test = df_test["loan_status"].values

In [23]:
#evaluate the model 

In [24]:
def evaluateModel(model, X_train, y_train, X_test, y_test):
    # FIT THE MODEL
    model.fit(X_train, y_train)

    # EVALUATE the MODEL
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    # PRINT the METRICS
    print("TRAINING SET")

    class_report_train = classification_report(y_train, train_preds)
    conf_matrix_train = confusion_matrix(y_train, train_preds)

    print(class_report_train)
    print(conf_matrix_train)

    print()

    # PRINT the METRICS
    print("Testing SET")

    class_report_test = classification_report(y_test, test_preds)
    conf_matrix_test = confusion_matrix(y_test, test_preds)

    print(class_report_test)
    print(conf_matrix_test)
    
    y_pred_proba = model.predict_proba(X_test)[::,1]
    fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)

    plt.figure(figsize=(10,6))
    plt.plot(fpr,tpr,label="auc="+str(auc))
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve Test Set")
    plt.legend(loc=4)
    plt.show()
    
    return(model)

In [25]:
My prediction on scaled data:
I believe that my RandomForestClassifier will preform better than my LogisticRegression since it can better accurately divide my data set based on catagorical values

SyntaxError: invalid syntax (Temp/ipykernel_24280/1384523154.py, line 1)

In [ ]:
# Initialize the model
lr = LogisticRegression()
lr = evaluateModel(lr, X_train, y_train, X_test, y_test)

In [ ]:
#randomforest
# Specify my X and Y
#drop target
X_train = df_train.drop(["loan_status"], axis=1).values
y_train = df_train["loan_status"].values

X_test = df_test.drop(["loan_status"], axis=1).values
y_test = df_test["loan_status"].values

In [ ]:
rf = RandomForestClassifier(random_state=42)
rf = evaluateModel(rf, X_train, y_train, X_test, y_test)

In conclusion LogisticRegression preforms better in scaled data and RandomForestClassifier is not affected much by scaled or unscaled data.